In [9]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np

from scipy.signal import butter, lfilter, freqz
from scipy import signal
data_after_applying_filter = []

cutoff = 25e3 #in Hz
fs = 114e3 
order = 5
NOISE_SIZE = 250
ECHO_SIZE = 512
THRESHOLD = 0.15

def butter_lowpass(cutoff, fs, order):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order):
    b, a = butter_lowpass(cutoff, fs, order=order)
    #y = lfilter(b, a, data)
    y = signal.filtfilt(b, a, data)
    return y

In [33]:
#Step 1 and 2
def get_time_domain_without_offset(filename):
    data_frame = pd.read_csv(filename, skiprows=[0], header= None)
    required_data_frame = data_frame.iloc[:, 9:]
    #print(data_frame)
    #print(required_data)
    required_data_without_offset = required_data_frame.sub(required_data_frame.mean(axis=1), axis=0).values
    return required_data_without_offset

#Step 3
def use_low_pass_filter(data_values):
    data_after_applying_filter = []
    for i, data in enumerate(data_values):
        y = butter_lowpass_filter(data, cutoff, fs, order)
        result = data - y
        data_after_applying_filter.append(result)
    return data_after_applying_filter

# Step 4:
def peak_value(data):
    max_point_distance = 0
    peakData = 0
    max_point_distance = np.array(data).argmax()
    peakData = np.array(data).max()
    if peakData > THRESHOLD:
        return max_point_distance
    else: 
        return None
    
def get_echos(filtered_values):
    all_echo_range = [] 
    for index, data in enumerate(filtered_values):
        chopped_data = data[NOISE_SIZE:]
        max_point_distance = peak_value(chopped_data)
        if max_point_distance:
            cutting_distance = max_point_distance - 200
            if cutting_distance > 0:
                echo_range = chopped_data[cutting_distance:]
                echo_range = echo_range[:ECHO_SIZE]
                all_echo_range.append(echo_range)
    return all_echo_range

def save_to_csv(echo_set, folder, file):
    data = pd.DataFrame(echo_set)
#     data.drop([0], axis=0)
    df = data.drop([17,19,20,21,22,23,36,28,29,31,32,43,44,45,46,47,48,49,50,51,53,55,56,57,58,59,
61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,86,87,88,90,91,93,96,97,98,99,100,104,105,107,108])
    print(folder,file)
    df.to_csv('./Documents/data_set/Result_without_filter/{}/{}_overall.csv'.format(folder, file), header=False, index=False)

In [34]:
file_set=[1]
distance_set = [110]

main_folder_set = ['Human']
folder_set = ['Human_H']
sub_folder_set = ['Human']

for i, folder_name in enumerate(main_folder_set):
    main_folder_name = main_folder_set[i]
    
    for i, sub_folder_name in enumerate(folder_set):
        file_name = sub_folder_set[i]
    
    for distance in distance_set:
        echo_set = []
        for file_number in file_set:
            filename = './Documents/data_set/New_Data_Set/{}/{}/{}_{}/{}.csv'.format(main_folder_name, sub_folder_name,file_name, distance, file_number)
            print(filename)
            time_domain_data_without_offset = get_time_domain_without_offset(filename)
#             filtered_values = use_low_pass_filter(time_domain_data_without_offset)
            echos_data =  get_echos(time_domain_data_without_offset)
            echo_set = echo_set + echos_data
        save_to_csv(echo_set, sub_folder_name, distance)

./Documents/data_set/New_Data_Set/Human/Human_H/Human_110/1.csv
Human_H 110
